In [7]:
# Set your parameters
class_type = "GE"  # GE or CM

In [8]:
import numpy as np
import pandas as pd
df = pd.read_csv(f'{class_type}_raw.csv')

In [9]:
df = df[df['comments'] != 0]

In [10]:
import pandas as pd

df = df.drop_duplicates(subset=['name', 'course_name'])
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [11]:
df_ratings = df
df_ratings[:10]

,course_name,name,overall_grade,grade,difficulty,useful,comments
0,GEST1001,KUAN UN FONG,C+,B,B,B,40
1,GEST1001,LAO WUN LEI,C+,B-,C+,B-,45
2,GEST1001,YEUNG YAU LUN,B-,C,C,C+,3
3,GEST1001,JIAO YIFAN,B-,B,B-,B,4
4,GEST1001,PUN SAU TAK,D,D,D+,D,18
5,GEST1001,YEUNG HANG FAI,B-,B,B-,B,15
6,GEST1001,WANG QIAN,A,A,B+,A-,9
7,GEST1017,CHEANG WAI SAN / LU JIAHONG / WANG RUIBING,C+,B,B,C+,5
14,GEST1017,CHEANG WAI SAN / LU JIAHONG / UNG OI LAM CAROLINA,B,A-,A-,B+,7
15,GELH2001,PANG KAM YIU SUNNY,B-,A,A,A,1


In [13]:
import pandas as pd

df_schedule = pd.read_excel(f"{class_type}.xlsx", skiprows=3)
df_schedule = df_schedule.rename(columns={
    "Unnamed: 2": "Course Code",
    "Unnamed: 3": "Course Title",
    "Unnamed: 4": "Section",
    "Unnamed: 8": "Teacher Information",
    "Unnamed: 10": "Day",
    "Unnamed: 11": "Time From",
    "Unnamed: 12": "Time To"
})

df_schedule['Time'] = df_schedule['Day'] + ' ' + df_schedule['Time From'].astype(str) + '-' + df_schedule['Time To'].astype(str)

course_info = df_schedule.groupby(['Course Code', 'Teacher Information']).apply(
    lambda x: x[['Section', 'Time']].to_dict('records')
).to_dict()

result = []
for _, row in df_ratings.iterrows():
    key = (row['course_name'], row['name'])
    sections_info = course_info.get(key, [])
    
    if not sections_info:
        result.append({**row.to_dict(), 'section': None, 'time': None})
    else:
        for section_info in sections_info:
            result.append({
                **row.to_dict(),
                'section': section_info['Section'],
                'time': section_info['Time']
            })

df_expanded = pd.DataFrame(result)
df_expanded[:10]

C:\Users\admin\AppData\Local\Temp\ipykernel_22712\1076417013.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  course_info = df_schedule.groupby(['Course Code', 'Teacher Information']).apply(


,course_name,name,overall_grade,grade,difficulty,useful,comments,section,time
0,GEST1001,KUAN UN FONG,C+,B,B,B,40,3.0,MON 11:30-12:45
1,GEST1001,KUAN UN FONG,C+,B,B,B,40,3.0,THU 11:30-12:45
2,GEST1001,KUAN UN FONG,C+,B,B,B,40,4.0,TUE 13:00-14:15
3,GEST1001,KUAN UN FONG,C+,B,B,B,40,4.0,FRI 13:00-14:15
4,GEST1001,KUAN UN FONG,C+,B,B,B,40,5.0,TUE 14:30-15:45
5,GEST1001,KUAN UN FONG,C+,B,B,B,40,5.0,FRI 14:30-15:45
6,GEST1001,LAO WUN LEI,C+,B-,C+,B-,45,1.0,MON 13:00-14:15
7,GEST1001,LAO WUN LEI,C+,B-,C+,B-,45,1.0,THU 13:00-14:15
8,GEST1001,LAO WUN LEI,C+,B-,C+,B-,45,2.0,MON 14:30-15:45
9,GEST1001,LAO WUN LEI,C+,B-,C+,B-,45,2.0,THU 14:30-15:45


In [14]:
def merge_times(times):
    return ' & '.join(sorted(set(t for t in times if pd.notna(t))))

df_merged = df_expanded.groupby(['course_name', 'name', 'section']).agg({
    'overall_grade': 'first',
    'grade': 'first',
    'difficulty': 'first',
    'useful': 'first',
    'comments': 'first',
    'time': merge_times
}).reset_index()

df_merged = df_merged.sort_values(['course_name', 'name', 'section'])
df_merged = df_merged.reset_index(drop=True)
df_merged[:10]

,course_name,name,section,overall_grade,grade,difficulty,useful,comments,time
0,GEGA1000,"LIAO BOLUN, EDGAR",5.0,C,C,C,C,3,MON 08:30-09:45 & THU 08:30-09:45
1,GEGA1000,"LIAO BOLUN, EDGAR",6.0,C,C,C,C,3,MON 10:00-11:15 & THU 10:00-11:15
2,GEGA1000,"LIAO BOLUN, EDGAR",7.0,C,C,C,C,3,MON 13:00-14:15 & THU 13:00-14:15
3,GEGA1000,SPOONER PAUL BARRIE,8.0,C,C+,C-,C,77,FRI 08:30-09:45 & TUE 08:30-09:45
4,GEGA1000,SPOONER PAUL BARRIE,9.0,C,C+,C-,C,77,FRI 10:00-11:15 & TUE 10:00-11:15
5,GEGA1000,SPOONER PAUL BARRIE,10.0,C,C+,C-,C,77,FRI 13:00-14:15 & TUE 13:00-14:15
6,GEGA1000,WEN RUJIA,2.0,D+,C-,D+,C-,279,FRI 14:30-15:45 & TUE 14:30-15:45
7,GEGA1000,WEN RUJIA,3.0,D+,C-,D+,C-,279,FRI 16:00-17:15 & TUE 16:00-17:15
8,GEGA1000,WEN RUJIA,4.0,D+,C-,D+,C-,279,MON 16:00-17:15 & THU 16:00-17:15
9,GEGA1000,XIE JINGZHEN,1.0,B-,B,B+,C+,4,MON 14:30-15:45 & THU 14:30-15:45


In [15]:
df_merged['section'] = df_merged['section'].astype(int)

In [16]:
df = df_merged

In [18]:
grade_order = {
    'A+': 12, 'A': 11, 'A-': 10,
    'B+': 9, 'B': 8, 'B-': 7,
    'C+': 6, 'C': 5, 'C-': 4,
    'D+': 3, 'D': 2, 'D-': 1,
    'F': 0
}

def grade_to_numeric(grade):
    return grade_order.get(grade, -1) 

sorted_df = df.sort_values(
    by=['overall_grade', 'course_name', 'name'],
    key=lambda x: x.map(lambda y: grade_to_numeric(y) if x.name == 'overall_grade' else y),
    ascending=[False, True, True]
)

sorted_df[:10]

,course_name,name,section,overall_grade,grade,difficulty,useful,comments,time
36,GELH1001,CHEONG IM LAN,3,A,A,A,A,10,MON 14:30-15:45 & THU 14:30-15:45
37,GELH1001,CHEONG IM LAN,4,A,A,A,A,10,MON 16:00-17:15 & THU 16:00-17:15
40,GELH1007,LI ZIHAO,1,A,A,A,A,2,SAT 08:30-11:15
138,GEST2002,KUZMA STRELNIKOV,1,A,A,A,A,2,MON 11:30-12:45 & THU 11:30-12:45
35,GELH1000,SUN XIAOXUE,10,A-,A,A,A,1,MON 08:30-09:45 & THU 08:30-09:45
41,GELH1010,LEE TING MIEN,1,A-,A,A,A,1,FRI 08:30-09:45 & TUE 08:30-09:45
14,GEGA1006,ZHA CHENYANG,4,B+,A-,A-,A-,18,FRI 17:30-18:45 & TUE 17:30-18:45
42,GELH2001,JEREMY CENTENO DE CHAVEZ,1,B+,A,B+,B+,18,MON 10:00-12:45
43,GELH2001,JEREMY CENTENO DE CHAVEZ,2,B+,A,B+,B+,18,FRI 16:00-18:45
118,GEST1003,WAN LUNG YAM,1,B+,A-,A-,B+,6,MON 11:30-12:45 & THU 11:30-12:45


In [21]:
# save to csv
sorted_df.to_csv(f'{class_type}.csv', index=False)

In [22]:
# check
sorted_df[sorted_df["course_name"]=="GELH1000"]

,course_name,name,section,overall_grade,grade,difficulty,useful,comments,time
35,GELH1000,SUN XIAOXUE,10,A-,A,A,A,1,MON 08:30-09:45 & THU 08:30-09:45
25,GELH1000,CHEONG IM LAN,4,B-,B+,B,B+,69,FRI 10:00-11:15 & TUE 10:00-11:15
26,GELH1000,CHEONG IM LAN,5,B-,B+,B,B+,69,FRI 11:30-12:45 & TUE 11:30-12:45
27,GELH1000,CHEONG IM LAN,6,B-,B+,B,B+,69,FRI 14:30-15:45 & TUE 14:30-15:45
21,GELH1000,CHEANG WAI LEK,17,C+,B,B,B-,37,MON 17:30-18:45 & THU 17:30-18:45
22,GELH1000,CHEANG WAI LEK,18,C+,B,B,B-,37,FRI 17:30-18:45 & TUE 17:30-18:45
23,GELH1000,CHEANG WAI LEK,19,C+,B,B,B-,37,SAT 10:00-12:45
18,GELH1000,CHAN WAI CHU,14,C,C,C,C-,20,MON 10:00-11:15 & THU 10:00-11:15
19,GELH1000,CHAN WAI CHU,15,C,C,C,C-,20,SAT 10:00-11:15 & WED 10:00-11:15
20,GELH1000,CHAN WAI CHU,16,C,C,C,C-,20,SAT 11:30-12:45 & WED 11:30-12:45
